# Nanopore amplicon processing & annotation

# 1. dorado Basecalling

#### Download Dorado basecaller
- if necessary

In [ ]:
!wget https://cdn.oxfordnanoportal.com/software/analysis/dorado-0.8.1-linux-x64.tar.gz -O /mnt/NanoporeRawData/dorado.tar.gz
!tar -xvf /mnt/NanoporeRawData/dorado.tar.gz

#### Fast5 Files Conversion
- if necessary

In [ ]:
import os
import subprocess

# Define paths
raw_data_path = "/mnt/NanoporeRawData/"
fast5_path = os.path.join(raw_data_path, "fast5")
pod5_path = os.path.join(raw_data_path, "pod5")

# Function to check if the number of files match and none are empty
def check_pod5_files(fast5_path, pod5_path):
    fast5_files = [f for f in os.listdir(fast5_path) if f.endswith('.fast5')]
    pod5_files = [f for f in os.listdir(pod5_path) if f.endswith('.pod5')]

    # Check if counts match and ensure no empty files
    if len(fast5_files) != len(pod5_files):
        return False

    for pod5_file in pod5_files:
        if os.path.getsize(os.path.join(pod5_path, pod5_file)) == 0:  # Check if the file is empty
            return False

    return True

# Check if only the fast5 directory exists, and convert FAST5 to POD5
if os.path.exists(fast5_path) and not os.path.exists(pod5_path):
    print("FAST5 directory detected, converting to POD5...")

    # Install pod5 package if not already installed
    subprocess.run(["pip", "install", "pod5"], check=True)

    # Ensure the pod5 directory exists
    os.makedirs(pod5_path, exist_ok=True)

    # Convert FAST5 to POD5, using -o for output and -O for one-to-one mapping
    subprocess.run([
        "pod5", "convert", "fast5",
        "-o", pod5_path,  # Output directory for POD5 files
        "-O", fast5_path,  # Parent directory for input files
        fast5_path  # Input path
    ], check=True)

# Rerun conversion if checks fail
if not check_pod5_files(fast5_path, pod5_path):
    print("File count mismatch or empty POD5 files detected. Re-running conversion...")
    subprocess.run([
        "pod5", "convert", "fast5",
        "-o", pod5_path,
        "-O", fast5_path,
        fast5_path
    ], check=True)

print("POD5 conversion completed.")

## 1-1. Pod5 Files Basecalling
- basecall
- convert BAM to fastq
- generate NanoPlot

In [ ]:
import os
import subprocess

# Define paths
raw_data_path = "/mnt/NanoporeRawData/pod5"
basecall_data_path = "/mnt/Data/1_dorado"

# Ensure the output directory exists
os.makedirs(basecall_data_path, exist_ok=True)

# Run dorado basecaller and write BAM output directly
print("Running dorado basecaller...")
bam_file_path = os.path.join(basecall_data_path, "all.bam")
with open(bam_file_path, "w") as bam_file:
    subprocess.run([
        "dorado", "basecaller", "sup", raw_data_path
    ], stdout=bam_file, check=True)

# Convert BAM to FASTQ using bedtools
print("Converting BAM to FASTQ...")
subprocess.run([
    "bedtools", "bamtofastq",
    "-i", bam_file_path,
    "-fq", os.path.join(basecall_data_path, "all.fastq")
], check=True)

# Generate NanoPlot
print("Generating NanoPlot...")
subprocess.run([
    "NanoPlot",
    "--fastq", os.path.join(basecall_data_path, "all.fastq"),
    "-o", basecall_data_path
], check=True)

print("Dorado basecalling and analysis completed.")


# 2. NanoACT Demultiplexing & Processing

##### Rename SampleID
- if only 1 gene fragment

In [ ]:
import os
import csv


''' ===== Configuration ===== '''
project_name = ""  # Set this to your project name
sample_type = "DNA samples"  # Specify the sample type for filtering
column_for_replace = "Sample number" #or Name or Morph description
replace_name = True  # Set True to replace SampleID with Reference ID from reference.csv

''' File path settings '''
base_dir = f"/mnt/Data/{project_name}"  # Base directory containing the csv files
barcode_ID_file = os.path.join(base_dir, f"{project_name}.csv")
reference_file = os.path.join(base_dir, "reference.csv")
output_file = os.path.join(base_dir, f"{project_name}_modf.csv")


''' ===== Workflow ===== '''
# Initialize containers for data
barcode_IDs = []
reference_data = {}

# Load reference data if the file exists
if replace_name and os.path.exists(reference_file):
    # Read the reference file and extract relevant columns
    with open(reference_file, 'r') as ref_file:
        reader = csv.DictReader(ref_file)  # Assuming CSV file
        for row in reader:
            # Filter by the specified sample type
            if row['Sample type'] == sample_type:
                reference_data[row['PCR ID']] = row[column_for_replace]
    print(f"Loaded reference data for sample type: {sample_type}.")
else:
    print("reference.csv not found. Skipping sample replacement step.")

# Read the barcode_ID file
with open(barcode_ID_file, 'r') as barcode_file:
    reader = csv.DictReader(barcode_file)
    barcode_header = reader.fieldnames  # Save the header
    for row in reader:
        sample_id = row['SampleID']  # Original SampleID from the barcode file
        # Replace SampleID if reference data is available and matches
        if reference_data and sample_id in reference_data:
            row['SampleID'] = reference_data[sample_id]  # Replace with Reference ID
        barcode_IDs.append(row)  # Add modified row to the list

# Prepare output data
output_data = barcode_IDs  # Append modified barcode data

# Write to the new CSV file
with open(output_file, 'w', newline='') as output_csv:
    writer = csv.DictWriter(output_csv, fieldnames=barcode_header)
    writer.writeheader()
    writer.writerows(output_data)

print(f"File written to: {output_file}")

##### Rename SampleID
- if > 1 gene fragment with same SampleID

In [ ]:
import os
import csv


''' ===== Configuration ===== '''
project_name = ""  # Set this to your project name
sample_type = "DNA samples"  # Specify the sample type for filtering
column_for_replace = "Sample number"  # Column from reference file to replace SampleID
replace_name = True  # Set True to replace SampleID with Reference ID from reference.csv

''' File path settings '''
base_dir = f"/mnt/Data/{project_name}"  # Base directory containing the csv files
barcode_ID_file = os.path.join(base_dir, f"{project_name}.csv")
reference_file = os.path.join(base_dir, "reference.csv")
output_file = os.path.join(base_dir, f"{project_name}_modf.csv")


''' ===== Workflow ===== '''
# Initialize containers for data
barcode_IDs = []
reference_data = {}

trichophyton_genes = {"SQLE": "TrSQLE-F1_H1", "EF": "EF-DermF_H1", "ITS": "1389F_H1"}

if replace_name and os.path.exists(reference_file):
    with open(reference_file, 'r') as ref_file:
        reader = csv.DictReader(ref_file)
        for row in reader:
            if row['Sample type'] == sample_type:
                primer_f = row.get("1' PCR Primer_F", "")
                matched_gene_id = next((gene for gene, marker in trichophyton_genes.items() if primer_f == marker), None)
                modified_sample_id = f"{row[column_for_replace]}_{matched_gene_id}" if matched_gene_id else row[column_for_replace]
                reference_data[row['PCR ID']] = modified_sample_id
    print(f"Loaded reference data for sample type: {sample_type}.")
elif replace_name:
    raise FileNotFoundError(f"Reference file not found at {reference_file}. Please provide the correct path.")

with open(barcode_ID_file, 'r') as barcode_file:
    reader = csv.DictReader(barcode_file)
    barcode_header = reader.fieldnames
    for row in reader:
        sample_id = row['SampleID']
        modified_sample_id = reference_data.get(sample_id, sample_id)
        row['SampleID'] = modified_sample_id
        barcode_IDs.append(row)

with open(output_file, 'w', newline='') as output_csv:
    writer = csv.DictWriter(output_csv, fieldnames=barcode_header)
    writer.writeheader()
    writer.writerows(barcode_IDs)

print(f"File written to: {output_file}")

## 2-1. Load NanoAct

In [ ]:
import os

working_directory = os.getcwd()

# Change to home directory
os.chdir(os.path.expanduser("~"))

# Check if 'nanoACT' directory exists
if not os.path.exists("nanoACT"):
    # If not, clone the repository
    !git clone https://github.com/Raingel/nanoACT.git
    os.chdir(os.path.expanduser("~/nanoACT/"))
else:
    # If the directory exists, reset local changes and pull the latest updates
    os.chdir(os.path.expanduser("~/nanoACT/"))
    !git fetch --all > /dev/null 2>&1
    !git reset --hard origin/main > /dev/null 2>&1 # Force reset to the latest commit
    !git pull > /dev/null 2>&1

# Install requirements if necessary
"""
!pip install --upgrade pip
!pip install -r requirements.txt
"""

# Import nanoAct and initialize
from nanoact import nanoact
dumb = nanoact.NanoAct(TEMP = "/home/nanoACT/temp/")

# Change back to the original working directory
os.chdir(working_directory)

# Verify the current working directory
print(os.getcwd())

## 2-2. Processing sequences
- Quality filtering
- Demultiplexing
- Orientation
- Trimming artificial reads

In [ ]:
import os


''' ===== Configuration ===== '''
project_name = ""  # Set this to your project name
renamed = False

''' Analysis settings '''
input_format = "fastq"
output_format = "fastq" #輸出檔案的格式，預設為 'both'。可以是 fastq 或 fasta。'both' 代表同時輸出 fastq 和 fasta
mismatch_ratio_f = 0.1 #FwIndex容許的錯誤率，預設為0.15。例如barcode長度為20bp，則容許0.15*20=3bp的錯誤(edit distance)
mismatch_ratio_r = 0.1 #RvAnchor容許的錯誤率，預設為0.15

# Quality Filter Configuration
QSCORE = 9 #recommended 7-9
MIN_LEN = 800 #depends on the length of your reads
MAX_LEN = 2000 #depends on the length of your reads

# Demultiplexing Configuration
expected_length_variation = 0.75 #預期的read長度變異，預設為0.3。例如預期的read長度為300bp，則容許0.3*300=90bp的變異
search_range = 150 #搜尋barcode的範圍，預設為150bp。代表搜尋範圍為前150bp和後150bp
rvc_rvanchor = False #預設為'False'。'True'則程式執行reverse-complement。

# Orientation Correction Configuration
orientation_search_range = 500 #搜尋FwPrimer和RvPrimer的範圍，預設為200bp。代表搜尋範圍為前200bp和後200bp。

# Trim Reads Configuration
fw_offset = 0 #從距離找到的切除位點開始往後切除幾個bp，預設為0，可以是負數。例如fw_offset=-10，則從距離找到的切除位點開始往前切除10個bp
rv_offset = 0 #從距離找到的切除位點開始往前切除幾個bp，預設為0，可以是負數。例如rv_offset=-10，則從距離找到的切除位點開始往後切除10個bp
discard_no_match = False
check_both_directions = True
reverse_complement_rv_col = True
trimming_search_range = 200

# Clustering Configuration (mmseqs_cluster)
cluster_min_seq_id = 0.98
cluster_mode = 0
cov_mode = 0
kmer_length = 15
kmer_per_seq = 20
sensitivity = 8.5
min_read_num = 4
suppress_output = True #suppress_output=False will output all details of the clustering process. Use it when unknown error occurs.

# Consensus Configuration (mafft_consensus)
minimal_reads = 2  # minimal_reads for consensus
max_reads = -1 #max_reads: 設定最多的序列數量，-1 代表不限制。例如max_reads=100，則只會隨機取100個序列進行排比
adjustdirection = False


''' ===== Workflow ===== '''
def process_data(project_name):
    data_base_path = f"/mnt/Data/{project_name}"
    src_path_dorado = os.path.join(data_base_path, "1_dorado")
    des_path_nanofilt = os.path.join(data_base_path, "2_nanofilt")
    des_path_demultiplex = os.path.join(data_base_path, "3_demultiplex")
    des_path_orientation = os.path.join(data_base_path, "4_orientation")  # Orientation output folder
    des_path_trimmed = os.path.join(data_base_path, "5_trimmed")  # Trimming output folder
    des_path_mmseqs = os.path.join(data_base_path, "6_mmseqs")
    des_path_consensus = os.path.join(data_base_path, "7_consensus")
    if renamed:
        barcode_index_file = os.path.join(data_base_path, f"{project_name}_modf.csv")
    else:
        barcode_index_file = os.path.join(data_base_path, f"{project_name}.csv")


    # Step 1. Filter by Quality and Length
    filtered_fastq = dumb.qualityfilt(
        src = os.path.join(src_path_dorado, 'all.fastq'),
        des = des_path_nanofilt,
        name = 'all_qualityfilt.fastq',
        QSCORE = QSCORE,
        MIN_LEN = MIN_LEN,
        MAX_LEN = MAX_LEN
    )

    # Step 2. Demultiplexing
    demultiplexed = dumb.singlebar(
        src = os.path.join(des_path_nanofilt, 'all_qualityfilt.fastq'),
        des = des_path_demultiplex,
        BARCODE_INDEX_FILE = barcode_index_file,
        mismatch_ratio_f = mismatch_ratio_f,
        mismatch_ratio_r = mismatch_ratio_r,
        expected_length_variation = expected_length_variation,
        search_range = search_range,
        rvc_rvanchor = rvc_rvanchor,
        input_format = input_format,
        output_format = output_format
    )

    # Step 3. Orientation correction
    orientation = dumb.orientation(
        src = des_path_demultiplex,
        des = des_path_orientation,
        input_format = input_format,
        output_format = output_format,
        BARCODE_INDEX_FILE = barcode_index_file,
        FwPrimer = "FwPrimer",
        RvPrimer = "RvPrimer",
        search_range = orientation_search_range
    )

    # Step 4. Trim Reads
    trimmed = dumb.trim_reads(
        src = des_path_orientation,
        des = des_path_trimmed,
        BARCODE_INDEX_FILE = barcode_index_file,
        fw_col = "FwPrimer",
        rv_col = "RvPrimer",
        input_format = input_format,
        output_format = output_format,
        mode = "table",
        fw_offset = fw_offset,
        rv_offset = rv_offset,
        mismatch_ratio_f = mismatch_ratio_f,
        mismatch_ratio_r = mismatch_ratio_r,
        discard_no_match = discard_no_match,
        check_both_directions = check_both_directions,
        reverse_complement_rv_col = reverse_complement_rv_col,
        search_range = trimming_search_range
    )

    # Step 5. Clustering
    clustering = dumb.mmseqs_cluster(
        src = des_path_trimmed,
        des = des_path_mmseqs,
        min_seq_id = cluster_min_seq_id,
        cluster_mode = cluster_mode,
        cov_mode = cov_mode,
        k = kmer_length,
        kmer_per_seq = kmer_per_seq,
        s = sensitivity,
        min_read_num = min_read_num,
        suppress_output = suppress_output,
        input_format = input_format,
        output_format = "fasta"
    )

    # Step 6. Consensus
    consensus = dumb.mafft_consensus(
        src = des_path_mmseqs,
        des = des_path_consensus,
        minimal_reads = minimal_reads,
        max_reads = max_reads,
        adjustdirection = adjustdirection,
        input_format = "fas"
    )

    return "Data processing complete."

# Call the function with the desired project ID:
result = process_data(project_name)
print(result)

# 3. Sequences Processing

## 3-1. Generate sequences processing summary

In [ ]:
import os
import csv
from Bio import SeqIO
import re
from itertools import groupby
from operator import itemgetter

''' ===== Configuration ===== '''
project_name = ""

'''File Path Settings'''
base_dir = f"/mnt/Data/{project_name}"
input_dir = os.path.join(base_dir, "7_consensus")
output_csv = os.path.join(base_dir, "con_fastq_summary.csv")

''' ===== Functions ===== '''
def parse_fas_files(input_dir):
    parsed_data = []
    for filename in os.listdir(input_dir):
        if filename.endswith('.fas') and filename.startswith('con_'):  # Check if filename starts with 'con_'
            file_path = os.path.join(input_dir, filename)
            for record in SeqIO.parse(file_path, "fasta"):
                fasta_name = record.id.split('_')[0]
                seq_length = len(record.seq)
                parsed_data.append((fasta_name, filename, seq_length))
    return parsed_data

def extract_read_number(filename):
    match = re.search(r'_r(\d+)', filename)
    return int(match.group(1)) if match else float('inf')  # Use 'inf' for files without read numbers


''' ===== Workflow ===== '''
parsed_data = parse_fas_files(input_dir)
parsed_data.sort(key=lambda x: extract_read_number(x[1]), reverse = True)  # Sort by read number from filename
with open(output_csv, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['FASTA_name', 'Filename', 'Sequence_Length'])
    csv_writer.writerows(parsed_data)

print(f"Parsed {len(parsed_data)} records. Output saved to {output_csv}")

## 3-2. Select the highest read files

In [ ]:
import os
import shutil
import re
import time
import csv
from collections import defaultdict
from threading import Thread
from Bio import SeqIO


''' ===== Configuration ===== '''
project_name = ""

'''File Path Settings'''
base_dir = f"/mnt/Data/{project_name}"
input_dir = os.path.join(base_dir, "7_consensus")
output_dir = os.path.join(base_dir, "8_highest_read_per_sample")
output_csv = os.path.join(base_dir, "highest_fasta_summary.csv")


''' ===== Spinner Function ===== '''
def spinner():
    symbols = ['|', '/', '-', '\\']
    idx = 0
    while not stop_spinner:
        print(f"\rProcessing files... {symbols[idx % len(symbols)]}", end='', flush=True)
        idx += 1
        time.sleep(0.1)


''' ===== Functions ===== '''
def extract_highest_read_files(input_dir, output_dir):
    global stop_spinner
    stop_spinner = False

    # Start the spinner in a separate thread
    spinner_thread = Thread(target=spinner)
    spinner_thread.start()

    try:
        # Check if the output directory exists, if not, create it
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Regex to extract sample name, optional gene ID, cluster number, and read number
        pattern = re.compile(
            r'con_([^_]+)(?:_([^_]+))?_cluster_(\d+)_r(\d+)\.fas', re.IGNORECASE
        )

        # Dictionary to store the highest read file for each sample
        highest_read_files = defaultdict(lambda: ('', 0))

        # Scan through all the files in the input directory
        for file in os.listdir(input_dir):
            if file.endswith('.fas'):
                match = pattern.match(file)
                if match:
                    sample_name, gene_id, cluster_number, read_number = match.groups()
                    key = (sample_name, gene_id)  # Use both sample name and gene ID as the key
                    read_number = int(read_number)
                    if read_number > highest_read_files[key][1]:
                        highest_read_files[key] = (file, read_number)

        # Copy the highest read files to the output directory
        copied_files_count = 0
        for file, _ in highest_read_files.values():
            src = os.path.join(input_dir, file)
            dest = os.path.join(output_dir, file)
            shutil.copyfile(src, dest)
            copied_files_count += 1

        # Stop the spinner
        stop_spinner = True
        spinner_thread.join()

        # Clear the spinner and print a final message
        print(f"\rCopied {copied_files_count} highest read files to {output_dir}".ljust(80))
    except Exception as e:
        stop_spinner = True
        spinner_thread.join()
        print(f"\rError occurred: {e}".ljust(80))

def parse_fas_files(input_dir):
    parsed_data = []
    for filename in os.listdir(input_dir):
        if filename.endswith('.fas') and filename.startswith('con_'):  # Check if filename starts with 'con_'
            file_path = os.path.join(input_dir, filename)
            for record in SeqIO.parse(file_path, "fasta"):
                fasta_name = record.id.split('_')[0]
                seq_length = len(record.seq)
                parsed_data.append((fasta_name, filename, seq_length))
    return parsed_data

def extract_read_number(filename):
    match = re.search(r'_r(\d+)', filename)
    return int(match.group(1)) if match else float('inf')  # Use 'inf' for files without read numbers

''' ===== Workflow ===== '''
if __name__ == "__main__":
    stop_spinner = False
    extract_highest_read_files(input_dir, output_dir)
    parsed_data = parse_fas_files(output_dir)  # Parse from the output directory
    with open(output_csv, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['FASTA_name', 'Filename', 'Sequence_Length'])
        csv_writer.writerows(parsed_data)

    print(f"Parsed {len(parsed_data)} records. Output saved to {output_csv}")


# 4. BLAST ID for the highest read files
- NCBI BLAST
- Local BLAST

In [ ]:
import os


''' ===== Configuration ===== '''
project_name = ""
output_csv = "blast_ncbi.csv"

'''File Path Settings'''
base_dir = f"/mnt/Data/{project_name}"
input_dir = os.path.join(base_dir, "8_highest_read_per_sample")
output_dir = base_dir

''' ===== Workflow ===== '''
dumb.blast_2 (src = input_dir, #Input: 一個資料夾，資料夾中包含以SampleID為檔名的fasta檔案，例如 SampleID.fasta
              des = output_dir, #Output: 一個資料夾，程式會在該資料夾中輸出一個csv檔案，檔案
              name = output_csv, #name: 輸出檔案的檔名，預設為blast.csv。
              funguild = True, #funguild: 是否進行funguild的分析，預設為True。
              startswith = "con_", #startswith: 輸入fasta檔案的檔名所具有的開頭，預設為con_。
              input_format = "fas", #input_format: 輸入fasta檔案的格式，預設為fasta, 若為fas則為副檔名為fas的fasta檔案。
              query_range = (None, None), #`query_range`: 代表該序列要用來送去blast的區間。一般建議不要超過500 bp。過長除了會造成blast伺服器負擔過重外，由於blast的排序同時考慮coverage及similarity。過長
                                #的query將會導致高coverage但低similarity的hit排在前面，排擠掉中等coverage但高similarity的hit
                                #輸入值為tuple，範例： 假設序列為 AAATTTCCC
                                #query_range=(None,None)則代表完全不裁切
                                #query_range=(0,None)也代表完全不裁切
                                #query_range=(0,-1) 則代表從第0個位置(第1個 bp，程式上習慣從0開始計數)開始，到(不包含)倒數最後一個，實際query為 AAATTTCC
                                #query_range=(2,5) 則代表從第2個開始，到(不包含)第5個，實際query為 ATT
                                #query_range=(3,-3) 則代表從第3個開始，到(不包含)倒數第3個，實際query為 TTT
              batch = 10 #`batch`: This parameter is an integer indicating the number of sequences to blast at a time.
                      #The input sequences are divided into batches of size `batch`, and each batch is blasted separately.
                      #This is done to avoid overloading the NCBI BLAST server with too many requests at once.
             )

#將consensus序列進行blast，並生成一個csv檔，內包含每個序列的blast結果
#Input: A folder containing fasta files named in the specified format
#Format: con_{sampleID}_cluster_{number}_r{reads_number}
#e.g.: con_2523_cluster_1_r499
#Output: A csv file named {name} is saved in the {des} folder
# `funguild`: This parameter is a boolean value that indicates whether to perform a Funguild search or not.
#             Funguild is a web-based annotation tool that allows users to predict the ecological functions of fungal communities based on their taxonomic composition.
# `startswith`: This parameter is a string indicating the prefix that the input fasta file names should start with. It is used to filter out files that do not match the given prefix.
# `query_range`: 代表該序列要用來送去blast的區間。一般建議不要超過500 bp。過長除了會造成blast伺服器負擔過重外，由於blast的排序同時考慮coverage及similarity。過長
#                的query將會導致高coverage但低similarity的hit排在前面，排擠掉中等coverage但高similarity的hit
#                輸入值為tuple，範例： 假設序列為 AAATTTCCC
#                query_range=(None,None)則代表完全不裁切
#                query_range=(0,None)也代表完全不裁切
#                query_range=(0,-1) 則代表從第0個位置(第1個 bp，程式上習慣從0開始計數)開始，到(不包含)倒數最後一個，實際query為 AAATTTCC
#                query_range=(2,5) 則代表從第2個開始，到(不包含)第5個，實際query為 ATT
#                query_range=(3,-3) 則代表從第3個開始，到(不包含)倒數第3個，實際query為 TTT
# `batch`: This parameter is an integer indicating the number of sequences to blast at a time.
#          The input sequences are divided into batches of size `batch`, and each batch is blasted separately.
#          This is done to avoid overloading the NCBI BLAST server with too many requests at once.
